In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: auto; }</style>"))
# TODO need to ctrl-shift-I after that?? wtf?

In [ ]:
import my.emfit as emfit
emfit_df = emfit.dataframe()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
from bokeh.models import LinearAxis, Range1d

In [ ]:
from datetime import timedelta
from bokeh.models import ColumnDataSource as CDS
df = emfit_df

df = df[df['error'].isnull()]
df = df.set_index('date')

# print(df[:10])

# raise RuntimeError
# todo make index??
# mean = cs.rolling(f'{d}D').mean()

# TODO ugh. figure out how pandas determines the index type..
import pandas as pd
df.index = pd.to_datetime(df.index, utc=True)


# see https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#basic-tooltips
# TODO date is displayed as unix time
from bokeh.models import HoverTool

hover = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
    ],

    formatters={
        '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

def date_figure(**kwargs):
    f = figure(x_axis_type='datetime', plot_width=2000, **kwargs)
    f.add_tools(hover)
    return f

# TODO reuse functions for adding moving averages? Perhaps they should operate on dataframes to be framework independent?
p = date_figure()


# TODO https://github.com/bokeh/bokeh/blob/master/examples/app/sliders.py

# TODO if there are Nans, it fails to plot anything??

m7d  = df[['avg_hr']].rolling('7D' ).mean()
m30d = df[['avg_hr']].rolling('30D').mean()

p.scatter(x='date', y='avg_hr', source=CDS(data=df  ), color='blue' , legend_label='HR'              )
p.line   (x='date', y='avg_hr', source=CDS(data=m7d ), color='red'  , legend_label='HR (7  days avg)')
p.line   (x='date', y='avg_hr', source=CDS(data=m30d), color='green', legend_label='HR (30 days avg)')


p.extra_y_ranges = {'resp': Range1d(start=10, end=25)}
# Adding the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name='resp'), 'right')
p.line(x='date', y='respiratory_rate_avg', source=CDS(data=df), color='orange', legend_label='Respiration', y_range_name='resp')

dates = df.index
def add_daysoff(p, bottom=0, top=80):
    # TODO hmm I guess need boxannotation after all?
    # print(p.y_range.computed_range)
    mind = min(dates)
    maxd = max(dates)
    from datetime import timedelta
    day = timedelta(1)

    # TODO need to keep boundary (-0.5, 0.5)?
    import numpy as np
    days = list(np.arange(mind, maxd, day))
    # TODO make a dataframe with it??

    import pandas as pd
    days_2 = pd.date_range(start=mind, end=maxd, freq='D')
    days_2.name = 'date'

    import my.calendar.holidays as holidays

    col_df = pd.DataFrame(index=days_2, columns=['color'])
    def calc_color(row):
        # todo separate column, abstract away
        dt = row.name
        is_workday = holidays.is_working_day(dt)
        return 'blue' if is_workday else 'red'

    col_df['color'] = col_df.apply(calc_color, axis=1)  

    from datetime import timedelta
    p.vbar(source=CDS(col_df), x='date', width=timedelta(1), bottom=bottom, top=top, color='color', alpha=0.05)

add_daysoff(p)
p.legend.click_policy = 'hide'


# https://stackoverflow.com/a/56258632/706389
# TODO box annotation vs vbar??

show(p)

# TODO determine delta hr and plot 'dependence' on workdays?
# TODO or plot 'derivative'?
# TODO more frequent ticks?

# hrv = df[['hrv_morning', 'hrv_evening']]

# TODO filter non-nan values??
dfm7d  = df.rolling('7D' ).mean()
dfm30d = df.rolling('30D').mean()

# TODO some hrv evening points are right at zero? careful, maybe worth filtering..
p2 = date_figure(x_range=p.x_range)
p2.scatter(x='date', y='hrv_morning', source=CDS(data=df)    , color='green' , legend_label='HRV (morning)')
p2.line   (x='date', y='hrv_morning', source=CDS(data=dfm7d) , color='green' , legend_label='HRV (morning)  7d', line_dash='dashed')
p2.line   (x='date', y='hrv_morning', source=CDS(data=dfm30d), color='green' , legend_label='HRV (morning) 30d')
p2.scatter(x='date', y='hrv_evening', source=CDS(data=df    ), color='purple', legend_label='HRV (evening)')
p2.line   (x='date', y='hrv_evening', source=CDS(data=dfm7d ), color='purple', legend_label='HRV (evening)  7d', line_dash='dashed')
p2.line   (x='date', y='hrv_evening', source=CDS(data=dfm30d), color='purple', legend_label='HRV (evening) 30d')
add_daysoff(p2)

p2.legend.click_policy = 'hide'
show(p2)
# todo disable individual points by default?

def mins(tt):
    # TODO shit. would just use time() but time doesn't allow wrapping around 24 hours
    # and timedelta is not serializable by plotly...
    mm = tt.hour * 60 + tt.minute
    if tt.hour < 16:
        mm += 24 * 60
    # TODO also careful about utc..
    return mm


ints  = df[['sleep_start', 'sleep_end']].applymap(lambda dt: mins(dt.time()))
# todo maybe instead plot angled lines? then won't need messing with minutes at all? Although still useful to keep 
p3 = date_figure(x_range=p.x_range)
from datetime import time
mint = mins(time(22, 0))
maxt = mins(time(11, 0))
add_daysoff(p3, bottom=mint, top=maxt)
p3.vbar(source=CDS(ints), x='date', width=timedelta(1), bottom='sleep_start', top='sleep_end', color='black', alpha=0.1)


from bokeh.models import FuncTickFormatter, FixedTicker
p3.yaxis.ticker = FixedTicker(ticks=list(range(mint, maxt, 30)))
p3.yaxis.formatter = FuncTickFormatter(code="""
    var hh = Math.floor(tick / 60 % 24).toString()
    var mm = (tick % 60).toString()
    if (hh.length == 1) hh = "0" + hh;
    if (mm.length == 1) mm = "0" + mm;
    return `${hh}:${mm}`
""")

p3.legend.click_policy = 'hide'
show(p3)

In [ ]:
df

In [ ]:
import my.bluemaestro as bluemaestro
orig_df = bluemaestro.dataframe()

In [ ]:
import datetime
from bokeh.models import ColumnDataSource as CDS

df = orig_df
df = df.sort_index() # FIXME should be fixed in hpi? but also make it defensive
df = df[-40000:]

# TODO 'index must be monotonic??'

print(df[-5:])
m1d  = df[['temp']].rolling('1D' ).mean()

p = figure(x_axis_type='datetime', plot_width=2000)
p.scatter(x='dt', y='temp', source=CDS(data=df ), color='blue', legend_label='Temperature'         )
p.line   (x='dt', y='temp', source=CDS(data=m1d), color='red' , legend_label='Temperature (1d avg)')
# TODO plot frequency of data?? could be even automatic (akin to hpi doctor/stat)

show(p)
# TODO highlight different days?

# TODO hpi: I feel like points duplicate each other without being filtered...
# and probably across different export files?? a bit crap

day_count = df.groupby(df.index.date).count()
print(day_count[:10])

p = figure(x_axis_type='datetime', plot_width=2000)
p.vbar(x='index', top='temp', source=CDS(day_count), width=datetime.timedelta(days=1) * 0.95)
show(p)


In [ ]:
# TODO for temp, maybe heatplot would make more sense? E.g. x axis days y axis time of day. although discontinuity at 0:00 is gonna be annoying.
# could group by weeks?

In [ ]:
# TODO hmm, maybe update regression lines dynamically? Wonder if there is as callback in bokeh
# https://stackoverflow.com/q/53287715/706389 here some sample code
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
# regression tap? some callbacky thing
# http://holoviews.org/reference/streams/bokeh/regression_tap.html

# TODO dynamic regression? https://gist.github.com/darothen/b1538c6571b67ed610669314551d3356

xxx = df[['avg_hr', 'coverage']].dropna()

# normal.opts(size=10) * hv.Slope.from_scatter(normal)
sc = hv.Scatter((xxx['avg_hr'], xxx['coverage']))
# TODO shit. need to filter nans??
hv.Slope.from_scatter(sc).opts(color='red') * sc.opts(size=10)

In [ ]:
import pandas as pd
import hvplot.pandas  # noqa

from hvplot import scatter_matrix

# todo add holiday/non-holiday? or days from holiday? could be interesting
cols = ['coverage', 'avg_hr', 'hrv_change', 'bed_time', 'recovery', 'respiratory_rate_avg']
# todo reglines
df = emfit_df[cols]
print(df)
from holoviews import opts

# todo uhoh. I guess the other fields won't be available for hover?
hover = HoverTool(
    tooltips=[(x, f'@{x}') for x in cols],
    #formatters={
    #    '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
    #},

    # display a tooltip whenever the cursor is vertically in line with a glyph
    # not sure about t hat here
    # mode='vline'
)

# todo ugh. frame_width is a bit crap, but works at least to some extent

# NOTE: 'fake' color is due to scatter_matrix ignoring kwargs if there is no color
df['color'] = 'fake'
sm = scatter_matrix(df, c='color').opts(
    # todo reuse/determine width
    opts.Scatter  (tools=[hover, 'box_select', 'lasso_select', 'tap'], frame_width=400, frame_height=400), 
    opts.Histogram(tools=[hover, 'box_select', 'lasso_select', 'tap'], frame_width=400, frame_height=400),
    # opts.Layout(width=2000, height=2000),
)

# hm nice, so GridMatrix is just a dict
for k in sm.keys():
    xx, yy = k
    if xx == yy:
        continue
    # TODO definitely test nan behaviour for scatter_matrix...

    zz = sm[(xx, yy)]
    dd = zz.data[[xx, yy]].dropna()
    # TODO ugh. need to restrict the length??
    # todo need to add various regression properties
    sm[k] = zz * hv.Slope.from_scatter(hv.Scatter((dd[xx], dd[yy]))).opts(color='red') # todo hopefuly this overlays correctly?? not sure about nans, again

sm


# wow! * (same plot) vs + (different plots) is pretty clever!
# also I like how it doesn't commute so the 'first' plot wins the layout parameters?


# TODO add tools to do vertical projections?


# ugh. https://github.com/holoviz/hvplot/blob/ab43c4f68aa7e485326dea567a348b96d24ebf60/hvplot/plotting/scatter_matrix.py#L45 not sure if the needed parameters are passed?..

# gridmatrix code? https://github.com/holoviz/holoviews/blob/5deb4a7b7f04f989fcc39bb06e657e8b6dfb4ea2/holoviews/operation/element.py#L997

# TODO date slider? to hightligh changes over dates E.g. left-right
# TODO show dates on tooltips

In [ ]:
# some examples of server with interactive stuff? 
# https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/main.py

# regression? https://docs.bokeh.org/en/latest/docs/gallery/anscombe.html
# odd, wonder how it's guessing the regresison line...

# these for sleep intervals? https://docs.bokeh.org/en/latest/docs/gallery/stacked_area.html
# also I had some old jawbone plots I think...
# also existing emfit plots


# food plots (breakdown) https://docs.bokeh.org/en/latest/docs/gallery/bar_stacked.html


# TODO errors should be aggregated and displayed as a separate column? with ability to toggle


# TODO ok, plotly has some callbacks for lasso/rectangle selection https://plotly.com/python/v3/selection-events/
# with these, I could dynamically update the regression lines, I guess


# TODO some nice dynamic interactions https://docs.bokeh.org/en/0.10.0/docs/user_guide/interaction.html